In [4]:
!pip install -U keras
!pip install -U tensorflow-gpu

Requirement already up-to-date: keras in /opt/conda/lib/python3.6/site-packages (2.3.1)
  Using cached tensorflow_gpu-2.1.0-cp36-cp36m-manylinux2010_x86_64.whl (421.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl (20.2 MB)
Processing /home/jovyan/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63/wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)
Processing /home/jovyan/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd/gast-0.2.2-cp36-none-any.whl
Processing /home/jovyan/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6/termcolor-1.1.0-cp36-none-any.whl
  Using cached opt_einsum-3.2.0-py3-none-any.whl (63 kB)
  Using cached tensorflow_estimator-2.1.0-py2.py3-none-any.whl (448 kB)
  Using cached tensorboard-2.1.1-py3-none-any.whl (3.8 MB)
  Using cached astor-0.8.1-py2.

In [5]:
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from pickle import load
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
def loader(textfile):
    # Open the file as read-only
    file = open(textfile, 'r')
    # Actually read the file
    text = file.read()
    # Now that I have the data in my text variable, need to close out of the file
    file.close()
    return text

In [7]:
raw = loader('../data/sequence_reviews.txt')

In [8]:
data = raw.split('\n')

In [9]:
data[0]

'talk about wow if anyone ever'

## Need to convert the words in my data to integers in order to be used by the model

In [10]:
tokenizer = Tokenizer()

def word_to_int(lines):
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(data)
    # Need the total size of the vocabulary for our embedding layer in model
    vocab_size = len(tokenizer.word_index) + 1
    return sequences, vocab_size

In [11]:
sequence_data, vocab_size = word_to_int(data)

In [12]:
# remove last empty row if any
if len(sequence_data[-1]) == 0:
    sequence_data = sequence_data[:-1]

In [13]:
sequence_array = np.array(sequence_data)

In [14]:
X, y = sequence_array[:,:-1], sequence_array[:,-1]

In [15]:
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [16]:
# defining my model

model = Sequential(name="basic-lstm")
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation = 'softmax'))
print(model.summary())

Model: "basic-lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 50)             1882100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 100)            60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 37642)             3801842   
Total params: 5,834,842
Trainable params: 5,834,842
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
filepath="model/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=20)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
670274/670274 [==============================] - 291s 434us/step - loss: 6.9139 - accuracy: 0.0694
Epoch 2/20
670274/670274 [==============================] - 252s 375us/step - loss: 6.2916 - accuracy: 0.1122
Epoch 3/20
670274/670274 [==============================] - 258s 385us/step - loss: 5.9441 - accuracy: 0.1274
Epoch 4/20
670274/670274 [==============================] - 250s 373us/step - loss: 5.7082 - accuracy: 0.1378
Epoch 5/20
670274/670274 [==============================] - 257s 384us/step - loss: 5.5246 - accuracy: 0.1463
Epoch 6/20
670274/670274 [==============================] - 251s 374us/step - loss: 5.3666 - accuracy: 0.1538
Epoch 7/20
670274/670274 [==============================] - 255s 381us/step - loss: 5.2265 - accuracy: 0.1607
Epoch 8/20
670274/670274 [==============================] - 251s 374us/step - loss: 5.0986 - accuracy: 0.1675
Epoch 9/20
670274/670274 [==============================] - 256s 382us/step - loss: 4.9799 - accuracy: 0.1738
Epoch 10/2